In [2]:
import torch
torch.multiprocessing.set_start_method('spawn', force="True")
import torch.distributions as dist
import torch.multiprocessing as mp
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import matplotlib.pyplot as plt
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import pandas as pd
from tqdm import tqdm_notebook
import sklearn

import numpy as np

In [16]:
# Get your price data here

from env_stocks import StockEnv
env=StockEnv(api_key='CTq2aKvtCkPPgR4L_NFs',
            instruments=['AAPL','MSFT','GOOGL','INTC','CSCO','ORCL','SAP','ADBE','IBM','CRM'],start_date='2016-06-01',end_date='2018-03-01',re_download=True,commission_fee=0.0005)

index_train = env.cleaned_market_data.AAPL.index
data_train = env.cleaned_market_data.AAPL.adj_close
data_train_np = env.cleaned_market_data.AAPL.values
data_norm_train = ((data_train - np.mean(data_train)) / (np.std(data_train) + 1e-5))

price_data = pd.DataFrame(data_train,index=index_train)
#price_data = dict(price_data)
#price_data = price_data.to_xarray()


Start to download market data
Done


/home/linyi/PortfolioManagement_Graduate/Version2/env_stocks.py:29: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  self.preprocessed_market_data, self.cleaned_market_data = self._init_market_data(re_download=re_download)
/home/linyi/test_project/venv/lib/python3.5/site-packages/pandas/core/generic.py:5382: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conver

In [11]:
# Define your network here

class NETWORK(nn.Module):
    def __init__(self, input_size = 1, rnn_layers=1, dp=0.2, batch_size=64):
        
        super(NETWORK, self).__init__()
        self.input_size = input_size
        self.rnn_layers = rnn_layers
        self.state_gru = nn.GRU(         # if use nn.RNN(), it hardly learns
            input_size=self.input_size,
            hidden_size=64,         # rnn hidden unit
            num_layers=self.rnn_layers,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        
        # Option: Can add batch normalization layer here
        
        #self.bn = nn.BatchNorm1d(501,momentum=0.5)
        #self.state_1 = nn.Linear(128,128)
        #self.bn = nn.BatchNorm1d(8,momentum=0.5)
        #self.state_2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 1) 
        
    def forward(self,x,hidden_state):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_state shape (n_layers, batch, hidden_size)    
        
        r_out,hidden_state = self.state_gru(x,hidden_state)
        #print(r_out.shape)
        #r_out = self.bn(r_out)
        outs = []
        for time_step in range(r_out.size(1)):
            outs.append(self.out(r_out[:,time_step,:]))
    
        return torch.stack(outs, dim=1), hidden_state

In [20]:
# Generate your sequence data here
# For example, using 0-6 days' price to predict the 1-7 days' price;

def generate_data(index,price):
    training_length = 7
    train_x = []
    train_y = []

    for step,pointer in enumerate(index):
        next_date = index[step+training_length]
        price_data = price.loc[index[step]:next_date]
        train_x.append((price_data[:-1]))
        train_y.append((price_data[1:]))
        if step%50 == 0:
            print(step)
        if step==len(index)-8:
            break
    return train_x,train_y


In [21]:
train_x,train_y = generate_data(index_train,data_train)

0
50
100
150
200
250
300
350
400


In [ ]:
# Convert data type from numpy to tensor
train_x,train_y = np.asarray(train_x),np.asarray(train_y)
# Make sure your data size is divisible by batch size
train_x,train_y = train_x[:400],train_y[:400]

# Divide your dataset into a few bathces
import torch.utils.data as Data
train_x, train_y = torch.from_numpy(train_x).float().cuda(), torch.from_numpy(train_y).float().cuda()
train_x = torch.unsqueeze(train_x,2)
train_y = torch.unsqueeze(train_y,2)
train_dataset = Data.TensorDataset(train_x, train_y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=10, shuffle=False, num_workers=1)

In [ ]:
# Initialize gru network
rnn = NETWORK(input_size=1).cuda()

In [ ]:
# Train your network

%matplotlib inline
hidden_state = None
LR = 1e-3
EPOCH = 100
batch_size=10
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.MSELoss()

for epoch in tqdm_notebook(range(EPOCH)):
    predictions = []
    total_loss = 0
    for step,(b_x,b_y) in enumerate(train_loader):
        #print(b_x.shape)
        prediction, hidden_state = rnn(b_x, hidden_state)
        predictions.append(prediction)

        hidden_state = hidden_state.cuda().data
        loss = loss_func(prediction,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
    print('Epoch: ', epoch, 'Train Loss: ', total_loss)
    predictions = torch.stack(predictions,dim=0)

In [ ]:
# Generate your test data again, repeat the steps before and test your data.

def generate_test_data(index,price):
    training_length = 7
    train_x = []
    train_y = []

    for step,pointer in enumerate(index):
        next_date = index[step+training_length]
        price_data = price.loc[index[step]:next_date]
        train_x.append((price_data[:-1]))
        train_y.append((price_data[1:]))
        if step%50 == 0:
            print(step)
        if step==len(index)-8:
            break
    return train_x,train_y

test_x,test_y = generate_data(index_test,data_test)

# Test
with torch.no_grad():
    
    prediction, h_state = rnn(test_x, h_state)   # rnn output
    # !! next step is important !!
    h_state = h_state.data   
    # plotting
    plt.plot(index_test[1:], test_y, 'r-')
    plt.plot(index_test[1:], prediction.cpu().data.numpy().flatten(), 'b-')
    plt.draw(); plt.pause(0.05)